# Desafío 03

## Integrantes

- Acevedo Zain, Gaspar (acevedo.zain.gaspar@gmail.com)

## Consignas

- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


***Sugerencias***
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


## Imports

In [72]:
import random
import io
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import pad_sequences
from scipy.special import softmax

In [54]:
import urllib.request
import bs4 as bs

## Selección del corpus

El objetivo de esta práctica es evaluar modelos de lenguajes con *tokenización por caracteres*, por lo cual, un texto lo suficientemente grande puede servir como `Corpus`.

Se elige entonces el libro `La Odisea` de Homero ([source](https://www.textos.info/homero/odisea/ebook)) como `Corpus`.

In [55]:
odisea_url = "https://www.textos.info/homero/odisea/ebook"

In [56]:
raw_html = urllib.request.urlopen(odisea_url)
raw_html = raw_html.read()

Se procesa el *html* original mediante la utilidad `bs.BeautifulSoup` a fin de tener el texto en ***un solo string***.

In [57]:
article_html = bs.BeautifulSoup(raw_html, 'lxml')
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text + ' '

article_text = article_text.lower()

Se muestran los primeros $500$ caracteres del texto (corpus).

In [58]:
article_text[:500]

' háblame, musa, de aquel varón de multiforme ingenio que, después de \ndestruir la sacra ciudad de troya, anduvo peregrinando larguísimo \ntiempo, vio las poblaciones y conoció las costumbres de muchos hombres y\n padeció en su ánimo gran número de trabajos en su navegación por el \nponto, en cuanto procuraba salvar su vida y la vuelta de sus compañeros a\n la patria. mas ni aun así pudo librarlos, como deseaba, y todos \nperecieron por sus propias locuras. ¡insensatos! comiéronse las vacas de\n helios'

En el texto se observan algunas *secuencias de escape de caracteres* (salto de línea o `\n`), por lo cual se las reemplaza a continuación con un caracter vacío.

In [59]:
article_text = str.replace(article_text, "\n", "")

In [60]:
article_text[:500]

' háblame, musa, de aquel varón de multiforme ingenio que, después de destruir la sacra ciudad de troya, anduvo peregrinando larguísimo tiempo, vio las poblaciones y conoció las costumbres de muchos hombres y padeció en su ánimo gran número de trabajos en su navegación por el ponto, en cuanto procuraba salvar su vida y la vuelta de sus compañeros a la patria. mas ni aun así pudo librarlos, como deseaba, y todos perecieron por sus propias locuras. ¡insensatos! comiéronse las vacas de helios, hijo '

## Definición del vocabulario + Tokenización

En esta sección definiremos nuestro `vocabulario` a partir del corpus original.

Luego lo tokenizaremos, a fin de que pueda ser procesado por una Red Neuronal en pasos posteriores.

Comenzamos definiendo nuestro vocabulario, que es el conjunto de distintos caracteres que aparecen en nuestro corpus.

Para el caso particular del texto seleccionado el tamaño es de $57$.

In [61]:
char_vocab = set(article_text)
print(f"Tamaño del vocabulario: {len(char_vocab)}")

Tamaño del vocabulario: 57


Definimos dos diccionarios que se utilizarán a lo largo de este trabajo:
- `char2idx`: A cada caracter de nuestro vocabulario se le asigna un `índice`.
- `idx2char`: Es el *inverso* de `char2idx`, es decir, dado un índice, me devuelve el caracter correspondiente.

In [62]:
char2idx = {k: v for v,k in enumerate(char_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

Ahora `tokenizamos` el corpus. Para ello hacemos uso del diccionario `char2idx`, reemplazando cada caracter por su índice correspondiente.

In [63]:
tokenized_text = [char2idx[ch] for ch in article_text]

Para validar, mostramos primero los $10$ primero caracteres tokenizados del corpus/texto, correspondientes a **" háblame, "** (nótesen los espacios en blanco al inicio y al final):

In [64]:
tokenized_text[:10]

[15, 49, 45, 50, 18, 33, 47, 34, 44, 15]

## Definición del dataset

En esta sección definimos los datos de entrenamiento y validación.

Primero se definien las siguientes constantes:
- `MAX_CONTEXT_SIZE`: corresponde al tamaño máximo del contexto que se analizará. Se define inicialmente en $100$.
- `VALIDATION_SIZE`: tamaño del set de validación. En este caso, se opta por utilizar el $30\%$.

In [65]:
MAX_CONTEXT_SIZE = 100
VALIDATION_SIZE = 0.3

Se define también la cantidad de secuencias de tamaño `MAX_CONTENT_SIZE` que tendrá el set de validación mediante la variable `SEQ_VALIDATION`.

Dado a que el tamaño del texto tokenizado es $673064$, el valor de `SEQ_VALIDATION` queda en $2020$, es decir, habrá en el set de validación un total de $2020$ secuencias de tamaño máximo $100$.

In [66]:
SEQ_VALIDATION = int(np.ceil(len(tokenized_text)*VALIDATION_SIZE/MAX_CONTEXT_SIZE))

Se realiza la separación del corpus original en `train` (train_text) y `validation` (val_text).

In [67]:
train_text = tokenized_text[:-SEQ_VALIDATION*MAX_CONTEXT_SIZE]
val_text = tokenized_text[-SEQ_VALIDATION*MAX_CONTEXT_SIZE:]

In [68]:
tokenized_sentences_val = [val_text[init*MAX_CONTEXT_SIZE:init*(MAX_CONTEXT_SIZE+1)] for init in range(SEQ_VALIDATION)]
tokenized_sentences_train = [train_text[init:init+MAX_CONTEXT_SIZE] for init in range(len(train_text)-MAX_CONTEXT_SIZE+1)]

In [69]:
X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])

Validamos los tamaños de `X` e `y`.

In [70]:
X.shape

(470964, 100)

In [71]:
y.shape

(470964, 100)

## Funciones útiles

En esta sección se definen una serie de funciones que se utilizarán a lo largo del entrenamiento de los distintos modelos a estudiar.

La función `PplCallback` fue tomada de las notebooks provistas por la materia (Fuente: [Clase 4 - 4_modelo_lenguaje_char.ipynb](https://github.com/gasper-az/procesamiento_lenguaje_natural/blob/main/clase_4/ejercicios/4_modelo_lenguaje_char.ipynb)).

La misma permite calcular la métrica de `Perplejidad` al final de cada *epoch* de *entrenamiento*.

Se caracteriza además por implementar `Early Stopping` en caso de que la métrica de perplejidad **NO** mejore luego de una cantidad definida de epochs (`patience`, con valore por defecto $5$).

También se encarga de guardar el *historial* del perplexity en cada epoch, a fin de poder analizar posteriormente.

In [76]:
class PplCallback(keras.callbacks.Callback):
    '''
    - Callback ad-hoc para calcular al final de cada epoch de entrenamiento
      la métrica de Perplejidad sobre un conjunto de datos de validación.
    - Implementa Early Stopping si la perplejidad no mejora después de
      `patience` epochs.
    '''

    def __init__(self, val_data, history_ppl, patience=5):
      self.val_data = val_data

      self.target = []
      self.padded = []
      self.history_ppl = history_ppl

      count = 0
      self.info = []
      self.min_score = np.inf
      self.patience_counter = 0
      self.patience = patience

      for seq in self.val_data:

        len_seq = len(seq)

        # armamos todas las subsecuencias
        subseq = [seq[:i] for i in range(1,len_seq)]
        self.target.extend([seq[i] for i in range(1,len_seq)])

        if len(subseq)!=0:
          self.padded.append(pad_sequences(subseq, maxlen=MAX_CONTEXT_SIZE, padding='pre'))
          self.info.append((count,count+len_seq))
          count += len_seq

      self.padded = np.vstack(self.padded)

    def on_epoch_end(self, epoch, logs=None):
        # Perplejidad de cada secuencia
        scores = []
        predictions = self.model.predict(self.padded,verbose=0)

        for start, end in self.info:
          # en `probs` iremos guardando las probabilidades de los términos target
          probs = [predictions[idx_seq,-1,idx_vocab] for idx_seq, idx_vocab in zip(range(start,end),self.target[start:end])]

          # calculamos la perplejidad por medio de logaritmos
          scores.append(np.exp(-np.sum(np.log(probs))/(end-start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores)
        self.history_ppl.append(current_score)
        print(f'\n mean perplexity: {current_score} \n')

        # Early Stopping
        if current_score < self.min_score:
          self.min_score = current_score
          self.model.save("my_model.keras")
          print("Saved new model!")
          self.patience_counter = 0
        else:
          self.patience_counter += 1
          if self.patience_counter == self.patience:
            print("Stopping training...")
            self.model.stop_training = True


Las siguientes funciones permiten hacer un `encode` y un `decode` de una secuencia dada.

Fueron tomadas también de las notebooks provistas por la materia. (Fuente: [Clase 4 - 4_modelo_lenguaje_char.ipynb](https://github.com/gasper-az/procesamiento_lenguaje_natural/blob/main/clase_4/ejercicios/4_modelo_lenguaje_char.ipynb)).

In [77]:
def encode(text, max_length=MAX_CONTEXT_SIZE):

    encoded = [char2idx[ch] for ch in text]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

    return encoded

def decode(seq):
    return ''.join([idx2char[ch] for ch in seq])

La función `generate_seq` permite, dado un modelo entrenado, generar una secuencia de caracteres. Es decir, funciona de manera autoregresiva.

Fue tomada también de las notebooks provistas por la materia. (Fuente: [Clase 4 - 4_modelo_lenguaje_char.ipynb](https://github.com/gasper-az/procesamiento_lenguaje_natural/blob/main/clase_4/ejercicios/4_modelo_lenguaje_char.ipynb)).

In [78]:
def generate_seq(model, seed_text, max_length, n_words):
    """
        Exec model sequence prediction

        Args:
            model (keras): modelo entrenado
            seed_text (string): texto de entrada (input_seq)
            max_length (int): máxima longitud de la sequencia de entrada
            n_words (int): números de caracteres a agregar a la sequencia de entrada
        returns:
            output_text (string): sentencia con las "n_words" agregadas
    """
    output_text = seed_text
	# generate a fixed number of words
    for _ in range(n_words):
		# Encodeamos
        encoded = [char2idx[ch] for ch in output_text.lower() ]
		# Si tienen distinto largo
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

		# Predicción softmax
        y_hat = np.argmax(model.predict(encoded,verbose=0)[0,-1,:])
		# Vamos concatenando las predicciones
        out_word = ''

        out_word = idx2char[y_hat]

		# Agrego las palabras a la frase predicha
        output_text += out_word
    return output_text

Las siguientes funciones serán utilizadas al estudiar `BEAM` search.

Fueron tomadas también de las notebooks provistas por la materia. (Fuente: [Clase 4 - 4_modelo_lenguaje_char.ipynb](https://github.com/gasper-az/procesamiento_lenguaje_natural/blob/main/clase_4/ejercicios/4_modelo_lenguaje_char.ipynb)).

In [79]:
def select_candidates(pred, num_beams, vocab_size, history_probs,
                      history_tokens, temp,mode):
  pred_large = []

  for idx,pp in enumerate(pred):
    pred_large.extend(np.log(pp+1E-10)+history_probs[idx])

  pred_large = np.array(pred_large)

  # criterio de selección
  if mode == 'det':
    # beam search determinista
    idx_select = np.argsort(pred_large)[::-1][:num_beams]
  elif mode == 'sto':
    # beam search con muestreo aleatorio
    idx_select = np.random.choice(np.arange(pred_large.shape[0]), num_beams,
                                  p=softmax(pred_large/temp))
  else:
    raise ValueError(f'Wrong selection mode. {mode} was given. det and sto are supported.')

  # traducir a índices de token en el vocabulario
  new_history_tokens = np.concatenate((np.array(history_tokens)[idx_select//vocab_size],
                        np.array([idx_select%vocab_size]).T),
                      axis=1)

  return pred_large[idx_select.astype(int)], new_history_tokens.astype(int)

In [80]:
def beam_search(model, num_beams, num_words, input, temp=1, mode='det'):
    encoded = encode(input)
    y_hat = model.predict(encoded,verbose=0)[0,-1,:]
    vocab_size = y_hat.shape[0]

    history_probs = [0]*num_beams
    history_tokens = [encoded[0]]*num_beams

    history_probs, history_tokens = select_candidates([y_hat],
                                        num_beams,
                                        vocab_size,
                                        history_probs,
                                        history_tokens,
                                        temp,
                                        mode)

    for i in range(num_words-1):
      preds = []
      for hist in history_tokens:
        input_update = np.array([hist[i+1:]]).copy()
        y_hat = model.predict(input_update,verbose=0)[0,-1,:]
        preds.append(y_hat)
      history_probs, history_tokens = select_candidates(preds,
                                                        num_beams,
                                                        vocab_size,
                                                        history_probs,
                                                        history_tokens,
                                                        temp,
                                                        mode)

    return history_tokens[:,-(len(input)+num_words):]

## Modelo Simple RNN